In [3]:
# Import Libraries and Dependencies

import os
import pandas as pd
import hvplot.pandas
import hvplot
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
import warnings

# Ignore any non-critical future warnings

warnings.filterwarnings("ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'alpaca_trade_api'

In [ ]:
# Load .env file

load_dotenv()

In [ ]:
# Set Alpaca API key and secret passwords

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [ ]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [ ]:
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what start date to begin data analysis

start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

In [ ]:
# Call stock data from identified period of time

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [ ]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

